In [1]:
# keras imports
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Input

# other imports
from sklearn.linear_model import LogisticRegression
import numpy as np
import os
import json
import pickle

In [2]:
config={
  "model"           : "vgg16",
  "weights"         : "imagenet",

  "classifier_path" : "C:/Users/ranji/OneDrive/Desktop/CAP/OUTCOMES/classifier.pickle",
  "img_p" : "C:/Users/ranji/OneDrive/Desktop/CAP/OCT2017/val/01-DME/dme8.jpeg",
    "val" : "C:/Users/ranji/OneDrive/Desktop/CAP/OCT2017/val/validation",

}

In [3]:
cur_path = config['val']
# load the trained logistic regression classifier
print ("[INFO] loading the classifier...")
classifier = pickle.load(open(config["classifier_path"], 'rb'))

# pretrained models needed to perform feature extraction on test data too!
if config["model"] == "vgg16":
	base_model = VGG16(weights=config["weights"])
	model = Model(base_model.input, base_model.get_layer('fc1').output)
	image_size = (224, 224)

else:
	base_model = None

[INFO] loading the classifier...


In [4]:
# Import necessary libraries
import pickle
import os
import glob
import numpy as np
import cv2
import keras


# Define the path to the validation dataset folder
validation_folder_path = 'C:/Users/ranji/OneDrive/Desktop/CAP/OCT2017/val'

# Initialize variables for keeping track of the number of correct and incorrect predictions
num_correct_predictions = 0
num_incorrect_predictions = 0

for image_path in glob.glob(cur_path + "/*.JPEG"):
    img = keras.utils.load_img(image_path, target_size=image_size)

    x = keras.utils.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feature = model.predict(x)
    flat = feature.flatten()
    flat = np.expand_dims(flat, axis=0)
    prediction = classifier.predict(flat)[0]
    
    ground_truth_label1 = image_path.split('_')[0]
    ground_truth_label = int(ground_truth_label1[-1])

    # Update the count of correct and incorrect predictions based on whether the predicted label matches the ground truth label
    if prediction == ground_truth_label:
        num_correct_predictions += 1
    else:
        num_incorrect_predictions += 1

# Calculate and print the accuracy
accuracy = num_correct_predictions / (num_correct_predictions + num_incorrect_predictions)
print(f'Accuracy: {accuracy}')


1/1 [==============================] - 0s 108ms/step
Accuracy: 0.96875


In [5]:
import keras
img_p = config["img_p"]
#read in the image, pre-proces the image and make predictions
img = keras.utils.load_img(img_p, target_size=image_size)
x = keras.utils.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
feature = model.predict(x)
flat = feature.flatten()
flat = np.expand_dims(flat, axis=0)
preds = classifier.predict(flat)

1/1 [==============================] - 0s 109ms/step


In [6]:
preds

array([2], dtype=int64)

In [7]:
label_check1 = ['normal','cnv', 'dme', 'drusen']

In [8]:
label_check1[preds[0]]

'dme'

In [9]:
import cv2

if preds!=0:
    import cv2

    # Load the retinal OCT scan image
    img = cv2.imread(img_p)

    # Get the dimensions of the image
    height, width, channels = img.shape

    # Define the region of interest (ROI) as the central 50% of the image
    roi_x = int(width * 0.25)
    roi_y = int(height * 0.25)
    roi_width = int(width * 0.5)
    roi_height = int(height * 0.5)

    # Crop the image to the ROI
    img_roi = img[roi_y:roi_y+roi_height, roi_x:roi_x+roi_width]

    # Convert the image to grayscale
    gray = cv2.cvtColor(img_roi, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to create a binary image with bright or dark spots
    _, binary = cv2.threshold(gray, 30, 255, cv2.THRESH_BINARY)

    # Find contours of the binary image
    contours, _ = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Find the contour with the largest area
    largest_contour = None
    largest_contour_area = 0
    for contour in contours:
        contour_area = cv2.contourArea(contour)
        if contour_area > largest_contour_area:
            largest_contour = contour
            largest_contour_area = contour_area

    # Draw the largest contour on the original image with green color and thickness of 2
    if largest_contour is not None:
        # Shift the contour to account for the ROI crop
        largest_contour[:, :, 0] += roi_x
        largest_contour[:, :, 1] += roi_y
    
        cv2.drawContours(img, [largest_contour], -1, (0, 255, 0), 2)

    # Show the resulting image
    cv2.imshow('Retinal OCT Scan with Highlighted Pathology', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
else:
    img = cv2.imread(img_p)
    cv2.imshow('Retinal OCT Scan with Highlighted Pathology', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    